In [6]:
!pip install unicodecsv
import unicodecsv

In [7]:
import unicodecsv

def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

enrollments = read_csv('/content/enrollments.csv')
daily_engagement = read_csv('/content/daily_engagement.csv')
project_submissions = read_csv('/content/project_submissions.csv')

print(enrollments[0])
print(daily_engagement[0])
print(project_submissions[0])

OrderedDict([('account_key', '448'), ('status', 'canceled'), ('join_date', '2014-11-10'), ('cancel_date', '2015-01-14'), ('days_to_cancel', '65'), ('is_udacity', 'True'), ('is_canceled', 'True')])
OrderedDict([('acct', '0'), ('utc_date', '2015-01-09'), ('num_courses_visited', '1.0'), ('total_minutes_visited', '11.6793745'), ('lessons_completed', '0.0'), ('projects_completed', '0.0')])
OrderedDict([('creation_date', '2015-01-14'), ('completion_date', '2015-01-16'), ('assigned_rating', 'UNGRADED'), ('account_key', '256'), ('lesson_key', '3176718735'), ('processing_state', 'EVALUATED')])


In [8]:
#replace acct column with account_key
for engagement_record in daily_engagement:
  engagement_record['account_key'] = engagement_record['acct']
  del[engagement_record['acct']] #delete "acct" once copied

# change data types

In [9]:
from datetime import datetime as dt

#takes string date and return data object
def str_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
#takes string returns int
def str_int(i):
    if i == '':
        return None
    else:
        return int(i)      

In [10]:
# change enrollments table data
for enrollment in enrollments:
    enrollment['cancel_date'] = str_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = str_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = str_date(enrollment['join_date'])
enrollments[0]


# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = str_date(engagement_record['utc_date'])
    
daily_engagement[0]



# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = str_date(submission['completion_date'])
    submission['creation_date'] = str_date(submission['creation_date'])

project_submissions[0]

OrderedDict([('creation_date', datetime.datetime(2015, 1, 14, 0, 0)),
             ('completion_date', datetime.datetime(2015, 1, 16, 0, 0)),
             ('assigned_rating', 'UNGRADED'),
             ('account_key', '256'),
             ('lesson_key', '3176718735'),
             ('processing_state', 'EVALUATED')])

In [11]:
'''
find the number of rows in the table and
the number of unique students in the table.
'''
enrollment_num_rows = len(enrollments)
print(enrollment_num_rows)

unique_enrolled_students = set()
for enrollment in enrollments:
    unique_enrolled_students.add(enrollment['account_key'])
enrollment_num_unique_students = len(unique_enrolled_students)
print(enrollment_num_unique_students)

engagement_num_rows = len(daily_engagement)
print(engagement_num_rows)

unique_engagement_students = set()
for engagement_record in daily_engagement:
    unique_engagement_students.add(engagement_record['account_key'])
engagement_num_unique_students = len(unique_engagement_students)
print(engagement_num_unique_students)

submission_num_rows = len(project_submissions)
print(submission_num_rows)

unique_project_submitters = set()
for submission in project_submissions:
    unique_project_submitters.add(submission['account_key'])
submission_num_unique_students = len(unique_project_submitters)
print(submission_num_unique_students)

1640
1302
136240
1237
3642
743


In [12]:
#find missing data
for enrollment in enrollments:
  student = enrollment['account_key']
  #if student from enrollment table is not present in engagement table
  if student not in unique_engagement_students: 
    print(enrollment)
    break

OrderedDict([('account_key', '1219'), ('status', 'canceled'), ('join_date', datetime.datetime(2014, 11, 12, 0, 0)), ('cancel_date', datetime.datetime(2014, 11, 12, 0, 0)), ('days_to_cancel', 0), ('is_udacity', False), ('is_canceled', True)])


In [13]:
error_rows = 0
for enrollment in enrollments:
  student = enrollment['account_key']
  #students present in enrollment but not in engagement 
  if student not in unique_engagement_students \
      and enrollment['join_date'] != enrollment['cancel_date']:
      error_rows += 1
      print(enrollment)
print(error_rows)   
'''
all 3 accounts have something common 
is_udacity = true 
means they are test accounts by udacity
'''

OrderedDict([('account_key', '1304'), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 1, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 3, 10, 0, 0)), ('days_to_cancel', 59), ('is_udacity', True), ('is_canceled', True)])
OrderedDict([('account_key', '1304'), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 3, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 6, 17, 0, 0)), ('days_to_cancel', 99), ('is_udacity', True), ('is_canceled', True)])
OrderedDict([('account_key', '1101'), ('status', 'current'), ('join_date', datetime.datetime(2015, 2, 25, 0, 0)), ('cancel_date', None), ('days_to_cancel', None), ('is_udacity', True), ('is_canceled', False)])
3


'\nall 3 accounts have something common \nis_udacity = true \nmeans they are test accounts by udacity\n'

# Find test student

In [14]:
'''
finding udacity test students
'''
test_ac = set()
for enrollment in enrollments:
  if enrollment['is_udacity']: #filter if test student
    test_ac.add(enrollment['account_key'])
len(test_ac)

6

#Delete test student

In [15]:
def remove_ac(data):
  non_udacity_ac = []
  for x in data:
    if x['account_key'] not in test_ac: # filter if student not in test_student set
      non_udacity_ac.append(x)
  return non_udacity_ac

In [16]:
non_udacity_enrollments = remove_ac(enrollments)
non_udacity_daily_engagement = remove_ac(daily_engagement)
non_udacity_project_submissions = remove_ac(project_submissions)

print(len(non_udacity_enrollments))
print(len(non_udacity_daily_engagement))
print(len(non_udacity_project_submissions))

1622
135656
3634


create dict of students who have not cancelled course
or engaged more than 7 days

In [17]:
paid_students = {}
for enrollment in non_udacity_enrollments:
  #filter students if status is not canceled or enrolled more than 7 days
  if (not enrollment['is_canceled'] or enrollment['days_to_cancel'] > 7):
      account_key = enrollment['account_key']
      enrollment_date = enrollment['join_date']
      #if ac key is not added or enrollment date is latest then add to dict 
      if (account_key not in paid_students or enrollment_date > paid_students[account_key]):
          paid_students[account_key] = enrollment_date
len(paid_students)

995

In [18]:
print(paid_students)

{'429': datetime.datetime(2015, 3, 10, 0, 0), '60': datetime.datetime(2015, 4, 1, 0, 0), '369': datetime.datetime(2014, 11, 10, 0, 0), '322': datetime.datetime(2015, 3, 10, 0, 0), '336': datetime.datetime(2014, 11, 10, 0, 0), '553': datetime.datetime(2014, 11, 10, 0, 0), '584': datetime.datetime(2015, 3, 10, 0, 0), '458': datetime.datetime(2015, 4, 6, 0, 0), '1058': datetime.datetime(2015, 3, 10, 0, 0), '206': datetime.datetime(2014, 11, 10, 0, 0), '45': datetime.datetime(2015, 3, 10, 0, 0), '315': datetime.datetime(2014, 11, 10, 0, 0), '604': datetime.datetime(2014, 11, 10, 0, 0), '204': datetime.datetime(2014, 11, 10, 0, 0), '408': datetime.datetime(2015, 4, 1, 0, 0), '51': datetime.datetime(2015, 3, 10, 0, 0), '323': datetime.datetime(2014, 11, 10, 0, 0), '568': datetime.datetime(2014, 11, 10, 0, 0), '130': datetime.datetime(2015, 4, 1, 0, 0), '550': datetime.datetime(2015, 5, 28, 0, 0), '44': datetime.datetime(2015, 3, 10, 0, 0), '198': datetime.datetime(2014, 11, 10, 0, 0), '368':

# Engagement of paid students within 1st week

remove students who cancelled after free trial


In [19]:
'''
if student is enrolled more than 7 days means 
student has paid for it
'''
def free_trial_cancel(data):
    new_data = []
    for x in data:
        if x['account_key'] in paid_students: #filter student if fees paid
            new_data.append(x)
    return new_data

paid_enrollments = free_trial_cancel(non_udacity_enrollments)
paid_engagement = free_trial_cancel(non_udacity_daily_engagement)
paid_submissions = free_trial_cancel(non_udacity_project_submissions) 

In [20]:
def one_week(join_date, engagement_date):
    time_diff = engagement_date - join_date
    ''' 
    if student started engagement within week of joining 
    and filter students who joined-cancelled-joined again.
    then engagement comes before joining and 
    that is an error
    '''
    return (time_diff.days < 7 and time_diff.days >=0)   

paid_engagement_first_week = []
for engagement_record in paid_engagement: #every student from paid list
    account_key = engagement_record['account_key']
    join_date = paid_students[account_key] # student's joining date
    engagement_record_date = engagement_record['utc_date'] #student's engagement date

    #filter if engagement is started within first 7 days
    if one_week(join_date, engagement_record_date): 
         paid_engagement_first_week.append(engagement_record)

len(paid_engagement_first_week)

6919

Finding data of each student

In [21]:
#doubt
'''
1)The functionality of both dictionaries and defualtdict are almost same 
but defualtdict never raises a KeyError 
It allows us to define a default value for the key that does not exists.
2)making groups by "account key"
'''
from collections import defaultdict

#if .... not present return empty list
engagement_account = defaultdict(list)

#each student from paid & engaged list
for record in paid_engagement_first_week:
  ac_key = record['account_key']
  engagement_account[ac_key].append(record)

sum all the entries for each account

In [22]:
#total min per student
total_min_student = {}

for account_key, engagement_of_student in engagement_account.items():
  total_min = 0 #init time with 0
  for x in engagement_of_student:
    total_min = total_min + x['total_minutes_visited']
  total_min_student[account_key] = total_min


avg of all acc


In [23]:
#avg of all min
import numpy as np
total_minutes = list(total_min_student.values()) #get only time not acc key
print("avg wathcing time:",np.mean(total_minutes))


avg wathcing time: 306.70832675342825


In [24]:
print("max wathcing time:",np.max(total_minutes))
print("min wathcing time:",np.min(total_minutes))
print("std deviation:",np.std(total_minutes))

max wathcing time: 3564.7332644989997
min wathcing time: 0.0
std deviation: 412.99693340852957
